In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification

In [4]:
key_genres = {
    "sci-fi": "Science Fiction",
    "science fiction": "Science Fiction",
    "scifi": "Science Fiction",
    "romcom": "Romance",
    "rom-com": "Romance",
    "doc": "Documentary",
    "bio": "Biopic",
    "biography": "Biopic",
    "war": "War",
    "thriller": "Thriller",
    "mystery": "Mystery",
    "crime": "Crime",
    "western": "Western",
    "fantasy": "Fantasy",
    "horror": "Horror",
    "comedy": "Comedy",
    "drama": "Drama",
    "action": "Action",
    "adventure": "Adventure",
    "animation": "Animation",
    "anime" : "Animation",
    "family": "Family",
    "music": "Music/Musical",
    "musical": "Musical/Musical",
    "history": "History",
    "sport": "Sport",
    "sports": "Sport",
    "romance": "Romance",
}

In [5]:
movies = pd.read_csv("movies-extended.csv")

# Text Cleanup

In [8]:
import re
sep_regex = re.compile(r"\s*[,/|;&]\s*")

In [9]:
def normalise_genre_token(token):
    t = token.strip().lower()
    if not t:
        return ""
    return key_genres.get(t, token.strip().title())

def parse_genre(cell):
    if not isinstance(cell, str):
        return []
    parts = sep_regex.split(cell)
    out = [normalise_genre_token(part) for part in parts if part.strip()]
    return sorted(list(dict.fromkeys(out)))

In [10]:
if "Plot" not in movies.columns:
    raise ValueError(f"Column 'Plot' not found in movies data! Columns available: {list(movies.columns)}")
if "Genre" not in movies.columns:
    raise ValueError(f"Column 'Genre' not found in movies data! Columns available: {list(movies.columns)}")

In [ ]:
movies['Plot'] = movies['Plot'].astype(str).fillna("").str.strip()
movies = movies[movies['Plot'].str.len() > 0].copy()

In [11]:
# Picking out all known genres vs unknown genres (simple negation below)
known_genres = movies['Genre'].astype(str).str.lower().ne('unknown')
unknown_genres = ~known_genres